In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

In [ ]:
y = df['Transported']
train = df.drop(columns=['Transported'])
train.head()

In [ ]:
y

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
numerical_cols = [col for col in train.columns if train[col].dtype in ['int64','float64']]
object_cols = [col for col in train.columns if train[col].dtype in ['object']]
object_cols.remove('PassengerId')
print(numerical_cols)
print(object_cols)

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer()
imp_train_num = pd.DataFrame(imp.fit_transform(train[numerical_cols]))
imp_test_num = pd.DataFrame(imp.fit_transform(test[numerical_cols]))
imp2 = SimpleImputer(strategy="most_frequent")
imp_train_obj = pd.DataFrame(imp2.fit_transform(train[object_cols]))
imp_test_obj = pd.DataFrame(imp2.fit_transform(test[object_cols]))

In [ ]:
imp_train_obj.columns = train[object_cols].columns
imp_train_num.columns = train[numerical_cols].columns

imp_test_obj.columns = test[object_cols].columns
imp_test_num.columns = test[numerical_cols].columns

In [ ]:
imp_train_num.head()

In [ ]:
imp_train_obj.isnull().sum()

In [ ]:
imp_train = pd.concat([imp_train_num, imp_train_obj], axis=1)
imp_train.head()

In [ ]:
imp_test = pd.concat([imp_test_num, imp_test_obj], axis=1)
imp_test.head()

In [ ]:
cols=['PassengerId']
train_pass_trans = train[cols]
imp_train = pd.concat([imp_train, train_pass_trans], axis=1)
imp_train.head()

In [ ]:
cols=['PassengerId']
test_pass = test[cols]
imp_test = pd.concat([imp_test, test_pass], axis=1)
imp_test.head()

In [ ]:
imp_train[object_cols].nunique()

In [ ]:
f_train = imp_train.drop(['Cabin','Name'],axis=1)
f_test = imp_test.drop(['Cabin','Name'],axis=1)
f_train.head()

In [ ]:
f_train.head()

In [ ]:
f_test.head()

In [ ]:
y

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

oe_cols = ['CryoSleep', 'VIP']
ohe_cols = ['Destination','HomePlanet']

ode = OrdinalEncoder()
ohe=OneHotEncoder(sparse=False,drop='first')

train_ode = pd.DataFrame(ode.fit_transform(f_train[oe_cols]))
test_ode = pd.DataFrame(ode.transform(f_test[oe_cols]))

train_ohe = pd.DataFrame(ohe.fit_transform(f_train[ohe_cols]))
test_ohe = pd.DataFrame(ohe.transform(f_test[ohe_cols]))

In [ ]:
train_ohe.columns = ohe.get_feature_names_out()
test_ohe.columns = ohe.get_feature_names_out()

In [ ]:
train_ohe.head()

In [ ]:
train_ode.columns = ode.get_feature_names_out()
test_ode.columns = ode.get_feature_names_out()

In [ ]:
train_ode.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder() #cannot decide order here, it will be randomly assigned
y_le = pd.DataFrame(le.fit_transform(y))

In [ ]:
y_le.columns = ['Transported']

In [ ]:
y_le.head()

In [ ]:
col_to_fetch = ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','PassengerId']
take_train = f_train[col_to_fetch]
take_test = f_test[col_to_fetch]

In [ ]:
X_train = pd.concat([train_ode, train_ohe,take_train], axis=1)
X_test = pd.concat([test_ode, test_ohe,take_test], axis=1)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
y_train = pd.DataFrame(y)
y_train.head()

In [ ]:
X_train_final = X_train.drop(['PassengerId'],axis=1)
X_test_final = X_test.drop(['PassengerId'],axis=1)

In [ ]:
X_train_final.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train_final,y_train)

In [ ]:
preds = clf.predict(X_test_final)

In [ ]:
preds

In [ ]:
output = pd.DataFrame({'PassengerId': X_test['PassengerId'],'Transported':preds})

In [ ]:
output.head()

In [ ]:
output.to_csv("submission.csv", index=False)